<p><a name="sections"></a></p>


# Data Exploration
</a><br>
- <a href="#import">Import Data</a><br>
- <a href="#eda">EDA Visualization</a><br>
- <a href="#processing">Data Processing</a><br>
    - <a href="#domain">Domain Knowledge Processing</a><br>
    - <a href="#missing">Missing Values</a><br>
    - <a href="#transform">Feature Transformation </a><br>
    - <a href="#numeric">Numeric Feature Processing </a><br>
    - <a href="#categorical">Categorical Feature Processing </a><br>
    - <a href="#outliers">Handling Outliers</a><br>
</a><br>

### Links:
- **Comprehensive data exploration with Python** https://www.kaggle.com/wordsforthewise/eda-with-python
- **Dataset 2007-2015 with more kernels** https://www.kaggle.com/wendykan/lending-club-loan-data
- **Google Docs** https://docs.google.com/document/d/1vutUT82n14cegW-OhpKH4A2d2qsPeS14ZJvxSPeK-rI/edit?usp=sharing


In [ ]:
import numpy as np
import pandas as pd

pd.options.display.max_columns = None
pd.options.display.max_rows = 100

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as matplotlib

%matplotlib inline

from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

# from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LinearRegression 
# from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler 

from scipy.stats import norm
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

<p><a name="import"></a></p>

## Import Data 

In [ ]:
# Importing data 

df_raw_accepted = pd.read_csv('accepted_2007_to_2018Q4.csv')
df_raw_rejected = pd.read_csv('rejected_2007_to_2018Q4.csv')

# df_raw_test = pd.read_csv('test.csv',index_col=0)
# df_raw = pd.concat([df_raw, df_raw_test], sort = False) 

In [ ]:
df_raw_accepted.shape

In [ ]:
df_raw_rejected.shape

In [ ]:
# 2007 - 2016 Approval Rate
1321847/11079386 

In [1]:
# 2007 - 2018 Approval Rate
2260701/(2260701+27648741)

0.07558486045978391

In [ ]:
#may want to sample according ratio  
#smote unbalance  

df_raw_accepted.sample(3000).to_csv('sample_accepted.csv')
df_raw_rejected.sample(3000).to_csv('sample_rejected.csv')


In [ ]:
sample_accepted = pd.read_csv('sample_accepted.csv')
sample_rejected = pd.read_csv('sample_rejected.csv')

In [ ]:
df_raw_accepted.describe()

In [ ]:
print(df_raw_accepted.columns)
print(df_raw_rejected.columns)

In [ ]:
print(df_raw_accepted.dtypes)
print(df_raw_rejected.dtypes)

### Question
- **How are the loan amounts/funded amounts distributed?** 
-loan amounts is the listed amount of the loan applied for by the borrower. Funded amounts is the total amount committed to that loan at that point in time. 4 out of 2000 samples, funded amounts is less than loan amount, the others are the same. Funded amounts distribution is right skewed, mean $14,233
- **Are there variations across different loan purposes, loan grades, etc?**
- **Are loans with higher funded amounts harder to be paid-in-full?**


In [ ]:
sample_accepted.describe()

In [ ]:
plt.hist(sample_accepted['loan_amnt'],bins=10)

In [ ]:
sns.distplot(sample_accepted['loan_amnt'],bins=10)

In [ ]:
plt.hist(sample_accepted['funded_amnt'])

In [ ]:
sample_accepted[sample_accepted['funded_amnt']!=sample_accepted['loan_amnt']]

In [ ]:
plotlist = ['term','grade','emp_length',
            #'purpose',
            'verification_status']

for column in plotlist:
    g = sns.FacetGrid(sample_accepted, col=column, hue=column, palette='Set1', size=10)
    g.map(sns.distplot, 'funded_amnt')

In [ ]:
sns.catplot(x="verification_status", y="funded_amnt", hue="grade", kind="bar", data=sample_accepted)

In [ ]:
sns.catplot(x="grade", y="funded_amnt", hue="term", kind="bar", data=sample_accepted)

In [ ]:
sns.catplot(x="grade", y="funded_amnt", hue="term", kind="bar", data=sample_accepted)

In [ ]:
sns.catplot(x="purpose", y="funded_amnt", kind="bar", data=sample_accepted, size=15)

In [ ]:
sns.catplot(x="grade", y="funded_amnt", hue="home_ownership", kind="bar", data=sample_accepted) 

In [ ]:
sample_accepted.loan_status.unique()

In [ ]:
large_amount = sample_accepted[sample_accepted.funded_amnt>15000]
small_amount = sample_accepted[sample_accepted.funded_amnt<15000]

In [ ]:
large_amount.shape

In [ ]:
large_amount[sample_accepted.loan_status == 'Fully Paid'].shape 

In [ ]:
204/743

In [ ]:
small_amount.shape

In [ ]:
small_amount[sample_accepted.loan_status == 'Fully Paid'].shape

In [ ]:
379/2261

In [ ]:
sns.countplot(x="term", hue=loan_status, data=large_amount)

In [ ]:
sns.catplot(x="loan_status", y="funded_amnt", kind="bar", data=sample_accepted, size = 20) 

### Question:
- For investors, the profitability of the loans is of their central 
concern. For a given loan, the profit-and-loss (in percentages) can be computed as
the (total_payment - principal)/principal.
- For those loans which are eventually 'Fully Paid', what are the average
returns (or the distributions of returns) of different loan grades/terms?
- For those loans wich are default or beyond, what are the average returns or return distributions?
- What about all the loans which have been 
terminated ('fully paid', 'default', 'charged off')?
- What about the loans which end up in **loan settlement negotiations**?
- Any variation of patterns for different loan purposes?
- What happens if the issuance years are included in your analysis?
- Is there any pattern between loan duration vs return rate?

In [ ]:
sample_accepted['loan_status'].unique()

In [ ]:
fully_paid = sample_return[sample_accepted['loan_status']=='Fully Paid']

In [ ]:
fully_paid.groupby('grade').aggregate({'return':'mean'})

In [ ]:
fully_paid.groupby('term').aggregate({'return':'mean'})

In [ ]:
fully_paid.groupby('sub_grade').aggregate({'return':'mean'})

In [ ]:
sample_return = sample_accepted.copy()
sample_return['return'] = sample_return['total_pymnt']/sample_return['total_rec_prncp'] - 1 

In [ ]:
charged_off = sample_return[sample_accepted['loan_status']=='Charged Off']

In [ ]:
charged_off.groupby('grade').aggregate({'return':'mean'})

In [ ]:
charged_off.groupby('term').aggregate({'return':'mean'})

In [ ]:
sample_return.groupby('loan_status').agg

In [ ]:
sample_accepted[sample_accepted.loan_status == 'Charged Off']

In [ ]:
sample_accepted[sample_accepted.loan_status == 'Default']

In [ ]:
df_raw_accepted[df_raw_accepted.loan_status == 'Default']

In [ ]:
1615/1321847

In [ ]:
accepted = df_raw_accepted[df_raw_accepted.loan_status == 'Fully Paid']

In [ ]:
accepted['return'] = accepted['total_pymnt']/accepted['total_rec_prncp'] - 1 

In [ ]:
accepted['return'].mean()

In [ ]:
plt.hist(accepted['return'],bins=20)

In [ ]:
# sample_accepted = sample_accepted[sample_accepted.loan_status == 'Fully Paid']
# sample_accepted['return'] = sample_accepted['total_pymnt']/sample_accepted['total_rec_prncp'] - 1 
# sample_accepted['return'].mean()

In [ ]:
plt.hist(sample_accepted['return'],bins=20)

 <p><a name="eda"></a></p>

## EDA Visualization

In [125]:
def getDistribution(df, primary_key, secondary_key):
    cnt = df[primary_key].value_counts()
    print('Count by primary key: ', primary_key)
    print(cnt)
    print('------------------------------------')
    df_count1 = pd.DataFrame(df.groupby([primary_key, secondary_key]).count().iloc[:, 0])
    print('Count by {} (primary key) and {} (secondary key)'.format(primary_key, secondary_key))
    print(df_count1)
    print('------------------------------------')
    df_cnt_unstk_resetIdx = df_count1.unstack().reset_index()
    for val in sorted(df[primary_key].unique()):
        df_cnt_unstk_resetIdx[df_cnt_unstk_resetIdx[primary_key] == val].plot.bar(title = val)

In [ ]:
df = sample_accepted
primary_key = 'purpose'
secondary_key = 'grade'
getDistribution(df, primary_key, secondary_key)

In [128]:
def getDistribution_numeric(df, primary_key, secondary_key):
    cnt = df[primary_key].unique()
    print('Count by primary key: ', primary_key)
    print(cnt)
    print('------------------------------------')
    df_count1 = pd.DataFrame(df.groupby([primary_key, secondary_key]).count().iloc[:, 0])
    print('Count by {} (primary key) and {} (secondary key)'.format(primary_key, secondary_key))
    print(df_count1)
    print('------------------------------------')   
    for val in sorted(df[primary_key].unique()):
        temp = df[df[primary_key] == val][[secondary_key]]
        plt.figure(figsize = (10, 8))
        _ = plt.hist(temp[secondary_key], bins = 20)
        plt.title(val, fontdict=None, loc='center')   

In [ ]:
df = df_raw_accepted
primary_key = 'purpose'
secondary_key = 'loan_amnt'
getDistribution_numeric(df, primary_key, secondary_key)

 <p><a name="processing"></a></p>

## Data Processing

- <a href="#domain">1.Domain Knowledge Processing</a><br>
- <a href="#outliers">2.Handling Outliers</a><br>
- <a href="#missing">3.Missing Values</a><br>
- <a href="#transform">4.Feature Transformation </a><br>
- <a href="#numeric">5.Numeric Feature Processing </a><br>

- <a href="#DM">6.Grouping Operations</a><br>
- <a href="#miss">7.Feature Split</a><br>
- <a href="#grouping">8.Scaling</a><br>
- <a href="#grouping">9.Extracting Date</a><br>


In [ ]:
# make a copy so the raw data will not be affected 
df_processed = sample_accepted.copy()

 <p><a name="1-40"></a></p>
 
### Columns 1- 40

In [ ]:
df_processed = df_processed.iloc[:,:41]

In [ ]:
# Drop irrelavant columns 
drop_list = ['Unnamed: 0','id','member_id','funded_amnt','url','desc','title']

drop_for_grade_list = ['funded_amnt_inv','int_rate','installment','issue_d','loan_status','pymnt_plan','out_prncp','out_prncp_inv']

df_processed = df_processed.drop(drop_list, axis=1)
df_processed = df_processed.drop(drop_for_grade_list, axis=1)

# Convert categorical to numerical 
df_processed['term'] = df_processed['term'].apply(lambda x: int(x.split()[0]))
df_processed['emp_length'] = df_processed['emp_length'].str.extract('(\d+)') 
#10 means more than 10 years 

# Convert to Datetime
df_processed['earliest_cr_line'] = pd.to_datetime(df_processed['earliest_cr_line'])

# Missing Values 

df_processed.mths_since_last_record = df_processed.mths_since_last_record.fillna(0)
df_processed.mths_since_last_delinq = df_processed.mths_since_last_delinq.fillna(0)

df_processed.emp_title = df_processed.emp_title.fillna('None')
df_processed.emp_length = df_processed.emp_length.fillna(0)

df_processed.revol_util = df_processed.revol_util.fillna(0)

df_processed.dti = df_processed.dti.fillna(df_processed.revol_bal / df_processed.annual_inc)

In [ ]:
df_processed.head(5)

In [ ]:
df_processed.dtypes

In [ ]:
#df_processed.emp_title.nunique()/len(df_processed.index)
#df_raw_accepted.emp_title.nunique() / len(df_raw_accepted.index)

<p><a name="missing"></a></p>

### Missing Values

In [ ]:
total = df_processed.isnull().sum().sort_values(ascending=False)
percent = (df_processed.isnull().sum()/df_processed.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

In [ ]:
df_processed.columns[df_processed.isnull().any()]

In [ ]:
# Missing Value Processing 
df_processed.mths_since_last_record = df_processed.mths_since_last_record.fillna(0)
df_processed.mths_since_last_delinq = df_processed.mths_since_last_delinq.fillna(0)

df_processed.emp_title = df_processed.emp_title.fillna('None')
df_processed.emp_length = df_processed.emp_length.fillna(0)
df_processed.revol_util = df_processed.revol_util.fillna(0)
df_processed.dti = df_processed.dti.fillna(df_processed.revol_bal / df_processed.annual_inc)

In [ ]:
np.sum(df_processed.isnull())

<p><a name="transform"></a></p>

### Feature Transformation 

In [ ]:
df_clean.SalePrice = np.log(df_clean.SalePrice)
df_clean.GrLivArea = np.log(df_clean.GrLivArea)
#df_clean.TotalBsmtSF = np.log(df_clean.TotalBsmtSF)
# after processing missing value 

In [ ]:
#histogram and normal probability plot
sns.distplot(df_raw['SalePrice'], fit=norm);
fig = plt.figure()
res = stats.probplot(df_raw['SalePrice'], plot=plt)

print("Skewness: %f" % df_raw['SalePrice'].skew())
print("Kurtosis: %f" % df_raw['SalePrice'].kurt())

In [ ]:
#histogram and normal probability plot
sns.distplot(df_clean['SalePrice'], fit=norm);
fig = plt.figure()
res = stats.probplot(df_clean['SalePrice'], plot=plt)

print("Skewness: %f" % df_clean['SalePrice'].skew())
print("Kurtosis: %f" % df_clean['SalePrice'].kurt())

In [ ]:
#histogram and normal probability plot
sns.distplot(df_raw['GrLivArea'], fit=norm);
fig = plt.figure()
res = stats.probplot(df_raw['GrLivArea'], plot=plt)

print("Skewness: %f" % df_raw['GrLivArea'].skew())
print("Kurtosis: %f" % df_raw['GrLivArea'].kurt())

In [ ]:
#histogram and normal probability plot
sns.distplot(df_clean['GrLivArea'], fit=norm);
fig = plt.figure()
res = stats.probplot(df_clean['GrLivArea'], plot=plt)

print("Skewness: %f" % df_clean['GrLivArea'].skew())
print("Kurtosis: %f" % df_clean['GrLivArea'].kurt())

In [ ]:
#histogram and normal probability plot
sns.distplot(df_raw['TotalBsmtSF'], fit=norm);
fig = plt.figure()
res = stats.probplot(df_raw['TotalBsmtSF'], plot=plt)

print("Skewness: %f" % df_raw['TotalBsmtSF'].skew())
print("Kurtosis: %f" % df_raw['TotalBsmtSF'].kurt())

# sns.distplot(df_clean['TotalBsmtSF'], fit=norm);
# fig = plt.figure()
# res = stats.probplot(df_clean['TotalBsmtSF'], plot=plt)

# print("Skewness: %f" % df_clean['TotalBsmtSF'].skew())
# print("Kurtosis: %f" % df_clean['TotalBsmtSF'].kurt())

<p><a name="numeric"></a></p>

### Numeric Feature Processing 

- scaler 
- normalization 


In [ ]:
var = 'YearBuilt'
data = pd.concat([df_raw['SalePrice'], df_raw[var]], axis=1)
f, ax = plt.subplots(figsize=(16, 8))
fig = sns.boxplot(x=var, y="SalePrice", data=data)
fig.axis(ymin=0, ymax=800000);
plt.xticks(rotation=90);

  
 
 ###  Categorical Features Processsing

In [ ]:
#box plot ordinal/saleprice
ord_list = ['Foundation','MasVnrType','Exterior1st','RoofStyle',
#             'RoofStyle', 'Neighborhood','LandSlope',
#             'LotConfig','Utilities','LotShape','LandContour',
#             'MSZoning','OverallQual','BsmtCond','Alley','BsmtExposure','BsmtFinType1','BsmtFinType2','BsmtQual','ExterCond','ExterQual','FireplaceQu','Functional','GarageCond','GarageQual','HeatingQC','KitchenQual','LandSlope','LotShape','PavedDrive','PoolQC',
            'Street']
for ordinal in ord_list:
    data = pd.concat([df_raw['SalePrice'], df_raw[ordinal]], axis=1)
    f, ax = plt.subplots(figsize=(8, 6))
    fig = sns.boxplot(x=ordinal, y="SalePrice", data=data)
    fig.axis(ymin=0, ymax=800000);
    

In [ ]:
# Neighborhood
df = df_clean[['SalePrice', 'GrLivArea', 'Neighborhood']]
df['UnitPrice'] = df_clean.SalePrice / df_clean.GrLivArea
df.groupby(['Neighborhood'])['UnitPrice'].agg({"UnitPrice":"mean"}).sort_values(['UnitPrice'])


In [ ]:
 #Encode some categorical features as ordered numbers when there is information in the order.
df_clean = df_clean.replace({"Alley" : {"None":0,"Grvl" : 1, "Pave" : 2},
                   "Neighborhood" : {"SWISU" : 1, "IDOTRR" : 2, "OldTown" : 3, "BrDale" : 4, "Blueste": 5,
                   "Edwards" : 6, "MeadowV" : 7, "BrkSide" : 8, "NWAmes" : 9, "NAmes" : 10, "NPkVill" : 11, "Sawyer" : 12, "Gilbert": 13, 
                   "SawyerW" : 14, "Crawfor" : 15, "ClearCr" : 16, "Mitchel" : 17, "NoRidge": 18, "Blmngtn" : 19, "CollgCr" : 20, "Timber" : 21, "Somerst" : 22, "Veenker": 23, "NridgHt" : 24, "StoneBr": 25 },
                   "BsmtCond" : {"None" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
                   "BsmtExposure" : {"None" : 0,"No":1, "Mn" : 2, "Av": 3, "Gd" : 4},
                   "BsmtFinType1" : {"None" : 0, "Unf" : 1, "LwQ": 2, "Rec" : 3, "BLQ" : 4,
                                     "ALQ" : 5, "GLQ" : 6},
                   "BsmtFinType2" : {"None" : 0, "Unf" : 1, "LwQ": 2, "Rec" : 3, "BLQ" : 4,
                                     "ALQ" : 5, "GLQ" : 6},
                   "BsmtQual" : {"None" : 0, "Po" : 1, "Fa" : 2, "TA": 3, "Gd" : 4, "Ex" : 5},
                   "ExterCond" : {"Po" : 1, "Fa" : 2, "TA": 3, "Gd": 4, "Ex" : 5},
                   "ExterQual" : {"Po" : 1, "Fa" : 2, "TA": 3, "Gd": 4, "Ex" : 5},
                   "FireplaceQu" : {"None" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
                   "Functional" : {"Sal" : 1, "Sev" : 2, "Maj2" : 3, "Maj1" : 4, "Mod": 5,
                                   "Min2" : 6, "Min1" : 7, "Typ" : 8},
                   "GarageCond" : {"None" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
                   "GarageQual" : {"None" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
                   "HeatingQC" : {"Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
                   "KitchenQual" : {"Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
                   "LandSlope" : {"Sev" : 3, "Mod" : 2, "Gtl" : 1},
                   "LotShape" : {"IR3" : 4, "IR2" : 3, "IR1" : 2, "Reg" : 1},
                   "PavedDrive" : {"N" : 0, "P" : 1, "Y" : 2},
                   "PoolQC" : {"None" : 0, "Fa" : 1, "TA" : 2, "Gd" : 3, "Ex" : 4},
                   "Street" : {"Grvl" : 1, "Pave" : 2}})

In [ ]:
# MSSubClass processing - MSSubClass 20-90 contains only duplicate information with HouseStyle and YearBuilt
# replace with 0 
'''
20	1-STORY 1946 & NEWER ALL STYLES
        30	1-STORY 1945 & OLDER
        40	1-STORY W/FINISHED ATTIC ALL AGES
        45	1-1/2 STORY - UNFINISHED ALL AGES
        50	1-1/2 STORY FINISHED ALL AGES
        60	2-STORY 1946 & NEWER
        70	2-STORY 1945 & OLDER
        75	2-1/2 STORY ALL AGES
        80	SPLIT OR MULTI-LEVEL
        85	SPLIT FOYER
        90	DUPLEX - ALL STYLES AND AGES
       120	1-STORY PUD (Planned Unit Development) - 1946 & NEWER
       150	1-1/2 STORY PUD - ALL AGES
       160	2-STORY PUD - 1946 & NEWER
       180	PUD - MULTILEVEL - INCL SPLIT LEV/FOYER
       190	2 FAMILY CONVERSION - ALL STYLES AND AGES

HouseStyle: Style of dwelling
       
       1Story One story
       1.5Fin One and one-half story: 2nd level finished
       1.5Unf One and one-half story: 2nd level unfinished
       2Story Two story
       2.5Fin Two and one-half story: 2nd level finished
       2.5Unf Two and one-half story: 2nd level unfinished
       SFoyer Split Foyer
       SLvl   Split Level
'''
# df_clean['MSSubClass'] = df_clean['MSSubClass'].replace([20,30,40,45,50,60,70,75,80,85], 0)
df_clean['MSSubClass'] = df_clean['MSSubClass'].replace(['20','30','40','45','50','60','70','75','80','85'], '0')

# convert numerical to categorical 
# df_clean[['MSSubClass','OverallQual','OverallCond']] = df_clean[['MSSubClass','OverallQual','OverallCond']].astype(str)
df_clean['MSSubClass'] = df_clean['MSSubClass'].astype(str)




# ordinal_list = ['OverallQual','OverallCond','ExterCond','BsmtQual','BsmtCond','HeatingQC','KitchenQual','Functional','FireplaceQu','GarageQual','PoolQC']

# enc = OrdinalEncoder()
# df_clean[ordinal_list] = enc.fit(df_clean[ordinal_list])

# OverallQual
# OverallCond
# ExterCond
# BsmtQual
# BsmtCond
# HeatingQC
# KitchenQual
# Functional
# FireplaceQu
# GarageQual
# PoolQC


## dummies 
df_clean = pd.get_dummies(df_clean, columns=df_clean.select_dtypes(include=['object']).columns, drop_first=True)



In [ ]:
# df_clean.select_dtypes(include=['object']).columns

In [ ]:
# df_clean.select_dtypes(include=['object'])

 <p><a name="domain"></a></p>
 
### Domain Knowledge Processing

 <p><a name="outliers"></a></p>
 
 ### Handling Outliers 


<p><a name="correlation"></a></p>


## Correlation

In [ ]:
#correlation matrix
corrmat = df_raw.corr()
f, ax = plt.subplots(figsize=(12, 9))
sns.heatmap(corrmat, vmax=.8, square=True);

In [ ]:
#saleprice correlation matrix
k = 10 #number of variables for heatmap
cols = corrmat.nlargest(k, 'SalePrice')['SalePrice'].index
cm = np.corrcoef(df_raw[cols].values.T)
sns.set(font_scale=1.25)
hm = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols.values, xticklabels=cols.values)
plt.show()

In [ ]:
#scatterplot
sns.set()
cols = ['SalePrice', 'OverallQual', 'GrLivArea', 'GarageCars', 'GarageArea','TotalBsmtSF', '1stFlrSF', 'FullBath', 'TotRmsAbvGrd', 'YearBuilt']
sns.pairplot(df_raw[cols], size = 5)
plt.show();

In [ ]:
#scatterplot
sns.set()
cols = [ 'OverallQual', 'GrLivArea', 'GarageArea','TotalBsmtSF', 'FullBath','SalePrice']
sns.pairplot(df_raw[cols], size =5)
plt.show();

In [ ]:
#histogram
sns.distplot(df_raw['SalePrice']);